In [4]:
import numpy as np
import sys
sys.path.append("..") 
import utils.funciones as fm
from IPython.display import clear_output

**DATOS INICIALES**

In [45]:
# HORIZOM
# VARIABLES
G_cnst = 4*(np.pi)**2  #(AU^3/(M_sun*año^2)))
dt_cnst = 0.0005
masa=np.array([
    1988410E24,     #Sol
    3.302E23,       #Mercurio
    48.685E23,      #Venus
    5.97219E24,      #Tierra
    6.4171E23,       #Marte
    18.9819E26,     #Jupiter
    5.6834E26,      #Saturno
    86.813E24,      #Urano
    102.409E24,     #Neptuno
    1.303E22        #Pluton
    ])/1988410E24

año_tropical=365.242190419
print("Año en días:", año_tropical)

#UA
posicion_t0=np.array([
    [-5.730606324378153E-03, -4.910557082772070E-03,  1.795094577858112E-04], #SOL   
    [-3.930336148500469E-01, -1.666347517169735E-01,  2.248723485902817E-02], #Mercurio   
    [4.476881591494200E-01,  5.573055222132830E-01, -1.826185079464067E-02], #Venus
    [-1.844140472974828E-01,  9.620722382946830E-01,  1.284152186349546E-04], #Tierra
    [-5.274164728925163E-01,  1.520324019719684E+00,  4.493510749539490E-02], #Marte
    [1.050302939252324E+00,  4.966541604941111E+00, -4.409855432676144E-02], #Jupiter
    [9.455336665176439E+00, -1.769525277925947E+00, -3.456968909591847E-01], #Saturno
    [1.109789820880128E+01,  1.608957335509925E+01, -8.401882312746477E-02], #Urano
    [2.987419674943718E+01, -6.390985521271113E-01, -6.753202855837556E-01], #Neptuno
    [1.822309610897269E+01, -3.001291441773554E+01, -2.059639830476407E+00], #Pluton
])

#UA/day
velocidad_t0=np.array([
    [7.160788611314778E-06, -3.666546272590425E-06, -1.176972196034231E-07], #SOL   
    [5.031590985068973E-03, -2.474711985703500E-02, -2.483041558403887E-03], #Mercurio   
    [-1.579911349143828E-02,  1.260639609552168E-02,  1.085100185838057E-03], #Venus  
    [-1.719757779305811E-02, -3.197199735579798E-03, -1.122400455364092E-07], #Tierra
    [-1.270467411479119E-02, -3.342506132667855E-03,  2.416611203580803E-04], #Marte
    [-7.469111612367896E-03,  1.920799528808175E-03,  1.591398359241618E-04], #Jupiter
    [7.165415690664376E-04,  5.471430990255198E-03, -1.233522942159990E-04], #Saturno
    [-3.266567450208665E-03,  2.049861782622937E-03,  4.996819331008718E-05], #Urano
    [4.657674111212642E-05,  3.156723229456242E-03, -6.648766294387875E-05], #Neptuno
    [2.767433185219602E-03,  9.357142006899497E-04, -9.006318946387802E-04], #Pluton
])*año_tropical

#Momento Lineal
momento_t0=masa[:,None]*velocidad_t0

Año en días: 365.242190419


# Inicialización del Modelo Físico: Correcciones de Centro de Masas

Para que un modelo de $N$-cuerpos sea físicamente coherente y numéricamente estable, debemos asegurar que el sistema no tenga una "deriva" artificial. Esto se logra trasladando el sistema al **Marco de Referencia del Centro de Masas**.

---

### 1. Corrección de la Posición
Si no se corrige la posición, el grupo de cuerpos puede estar muy lejos del origen $(0,0,0)$, lo que causa pérdida de precisión en los cálculos de punto flotante y dificulta la visualización.

Calculamos la posición del centro de masas $\vec{R}_{cm}$:
$$\vec{R}_{cm} = \frac{\sum_{i=1}^{n} m_i \vec{r}_i}{\sum_{i=1}^{n} m_i}$$

**Corrección aplicada:**
$$\vec{r}_{i, \text{nuevo}} = \vec{r}_i - \vec{R}_{cm}$$

> **Efecto:** El sistema queda perfectamente centrado en el origen, de modo que $\sum m_i \vec{r}_{i, \text{nuevo}} = 0$.

---

### 2. Corrección de la Cantidad de Movimiento (Momento)
Un sistema sin fuerzas externas debe tener un momento neto nulo para no desplazarse linealmente por el espacio. El momento neto inicial se calcula como:
$$\vec{P}_{neto} = \sum_{i=1}^{n} \vec{p}_i$$

Para corregirlo sin alterar la física relativa entre los cuerpos, restamos a cada partícula una fracción del momento total proporcional a su contribución de masa:
$$\vec{p}_{i, \text{nuevo}} = \vec{p}_i - \left( \frac{m_i}{M_{total}} \right) \vec{P}_{neto}$$

**¿Por qué proporcional a la masa?**
Porque esto equivale a restar la misma velocidad de deriva $\vec{V}_{cm}$ a todos los cuerpos:
$$\Delta \vec{v} = \frac{\vec{P}_{neto}}{M_{total}}$$
Esto garantiza que la estructura interna del sistema (velocidades relativas) se mantenga intacta.

---

### 3. Verificación de las Condiciones de Equilibrio
Tras las correcciones, el modelo debe satisfacer:
1. **Momento Neto Nulo:** $\sum \vec{p}_i \approx 0$
2. **Posición del CM en el origen:** $\vec{R}_{cm} \approx 0$

**CORRECCIÓN DE MOMENTO NETO**

In [50]:
masa_total = masa.sum()
print("Masa Total:", masa_total)

momento_neto = momento_t0.sum(axis=0)
print("Momento Neto:", momento_neto)

proporcion_de_masa = masa/masa_total

momento_t0 = momento_t0 - momento_neto*proporcion_de_masa[:,None]

print("Momento Neto Corregido:", momento_t0.sum(axis=0))

Masa Total: 1.0013415631735911
Momento Neto: [ 5.91483769e-20 -4.80765644e-20  2.00839453e-21]
Momento Neto Corregido: [ 5.91483769e-20 -4.80765644e-20  2.00839453e-21]


**CORRECIÓN DE POSICIÓN**

In [58]:
posicion_neta = (masa[:, None] * posicion_t0).sum(axis=0)

r_cm = posicion_neta / masa_total

print("Centro de Masas original:", r_cm)

posicion_t0 = posicion_t0 - r_cm

posicion_neta_corrg= (masa[:, None] * posicion_t0).sum(axis=0)

print("Centro de Masas corregido:", posicion_neta_corrg / masa_total)


Centro de Masas original: [-9.79655762e-20 -5.56442358e-20 -9.47174596e-21]
Centro de Masas corregido: [-9.79655762e-20 -5.56442358e-20 -9.47174596e-21]


**CREACIÓN DE VARIBLE**

In [59]:
#TIEMPO DE SIMULACIÓN
t=1000
simulaciones=int((t/dt_cnst))
print("Iteraciones:", simulaciones)

r_contenedor = np.zeros((simulaciones+1, 10 , 3))
p_contenedor = np.zeros((simulaciones+1, 10 , 3))

#Asignación de valores iniciales
r_contenedor[0, ...] = posicion_t0
p_contenedor[0, ...] = momento_t0

print(r_contenedor.shape)

Iteraciones: 2000000
(2000001, 10, 3)


Método simpléctico de Yoshida (cuarto orden)

El método de Yoshida de cuarto orden se construye como una composición de
integradores simplécticos de segundo orden:

$$
\Phi^{(4)}(h)
=
\Phi^{(2)}(w_1 h)
\circ
\Phi^{(2)}(w_0 h)
\circ
\Phi^{(2)}(w_1 h),
$$

donde los coeficientes están dados por

$$
w_1 = \frac{1}{2 - 2^{1/3}},
\qquad
w_0 = -\frac{2^{1/3}}{2 - 2^{1/3}}.
$$

Este esquema conserva la estructura simpléctica del sistema y presenta un
error global de orden

$$
\mathcal{O}(h^4).
$$


In [60]:
for i in range(simulaciones):
    r_contenedor[i+1,...], p_contenedor[i+1,...]=fm.Yoshida(r_contenedor[i,...],p_contenedor[i,...],masa,G_cnst,dt_cnst)
    if i%1000==0:
        clear_output(wait=True)
        print(f"Progreso: {((i+1)/(simulaciones))*100:.4f}%")
        
clear_output(wait=True)
print(f"Progreso: {100}%", "Exito")


Progreso: 99.9501%
Progreso: 100% Exito


In [62]:
np.savez("../resultados/posiciones.npz",pos=r_contenedor)
np.savez("../resultados/momentos.npz",mon=p_contenedor)

print("Se guardo correctamente")

Se guardo correctamente
